In [1]:
!pip install xlrd

In [ ]:
import pandas as pd

df = pd.read_excel('GSAF5.xls')
df.head(10)

In [ ]:
df.tail(10)

In [ ]:
df.sample(10)

In [ ]:
df.columns.to_list()

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df['Fatal Y/N'].value_counts()

----------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------------

RODRIGO CODE

----------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------------

Cleaning 'Fatal' Data Column:

In [ ]:
df['Fatal Y/N'] = df['Fatal Y/N'].astype(str).str.strip().str.upper()

df['Fatal Y/N'] = df['Fatal Y/N'].replace({
    'N ': 'N',
    ' N': 'N',
    'UNKNOWN': 'Unknown',  
    'F': 'Unknown',
    'Y ': 'Y',
    'M': 'Unknown', 
    '2017': 'Unknown',
    '2018': 'Unknown',
    '0': 'Unknown'
})

df.loc[~df['Fatal Y/N'].isin(['Y', 'N']), 'Fatal Y/N'] = 'Unknown'



In [102]:
df['Fatal Y/N'].value_counts()

Fatal Y/N
N          4924
Y          1483
Unknown     643
Name: count, dtype: int64

Adding clean 'Injury_Class' column to the data set:

In [ ]:
df['Injury'] = df['Injury'].fillna('Unknown').astype(str).str.lower().str.strip()

def classify_injury(text):
    text = str(text).lower().strip()
    if 'no injury' in text or 'uninjured' in text:
        return 'No Injury'
    elif 'fatal' in text or 'died' in text or 'death' in text:
        return 'Fatal Wounds'
    elif any(word in text for word in [
        'minor', 'scratch', 'abrasion', 'bruise', 'small', 'superficial',
        'cut', 'scrape', 'nipped', 'grazed', 'bitten but not serious'
    ]):
        return 'Minor Wounds'
    elif any(word in text for word in [
        'amputation', 'severe', 'major', 'deep', 'critical',
        'multiple', 'serious', 'massive', 'wound', 'bites', 'bitten',
        'injury to', 'injuries to', 'damage', 'laceration', 'torn'
    ]):
        return 'Severe Wounds'
    else:
        return 'Unknown'

df['Injury_Class'] = df['Injury'].apply(classify_injury)
df['Injury_Class'] = df['Injury_Class'].str.title()

In [ ]:
df['Injury_Class'].value_counts()

In [ ]:
df['Injury_Class'].head(10)

In [ ]:
df['Injury'].value_counts()

----------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------------

In [11]:
# Divya's work
# Cleaning up 'activity' column in the dataset

df['Activity'].nunique() #unique values in the activity column

1608

In [12]:
df['Activity'].unique()


array(['Swimming with sharks', 'Fishing/swimming', 'Surfing', ...,
       'Crew swimming alongside their anchored ship',
       '4 men were bathing', 'Wreck of  large double sailing canoe'],
      dtype=object)

In [13]:
df['activity_clean'] = (
    df['Activity']
    .str.lower() #lowercase
    .str.strip() #remove spaces
    .str.replace(r'[^a-z\s]', '', regex=True)  #remove punctuation/numbers
)


In [45]:
import numpy as np

def categorize_activity(text):
    if pd.isna(text): #handling 'NaN' values and labelling them as unknown
        return "unknown"
    if any(word in text for word in ["surf", "bodyboard", "paddle", "boogie", "body boarding"]): #if the word is present in the text, then loop over each word and replace with return 'string'
        return "surfing"
    elif any(word in text for word in ["swim", "bathing", "snorkel", "rescue", "float", "splash", "swimming"]):
        return "swimming"
    elif any(word in text for word in ["fish", "spearfish", "net", "catch", "line", "fishing", "spear"]):
        return "fishing"
    elif any(word in text for word in ["dive", "scuba", "freediv", "underwater", "research", "investigat", "pearl", "recover", "diving"]): #'investigat' covers terms like investigating / investigation / investigator..
        return "diving"
    elif any(word in text for word in ["boat", "kayak", "sail", "ship", "vessel", "frigate", "dinghy", "canoe", "race", "compet", "rowing", "watercraft", "jet ski"]):
        return "boating"
    elif any(word in text for word in ["walk", "stand", "wade", "reef", "shore", "beach", "adrift", "wading", "tread"]):
        return "wading"
    elif any(word in text for word in ["sea disaster", "aircraft", "boeing", "wreck", "hurricane", "tsunami", "earthquake", "disaster", "plunged", "sank", "destroyed", "overboard", "suicide", "air", "petting", "capsize", "swept", "help", "ride", "sunk", "went down", "crash"]):
        return "catastrophe"
    else:
        return "other"

df['activity_group'] = df['activity_clean'].apply(categorize_activity) #new column 'activity group' for categorised activity


In [15]:
print("Original:", df['Activity'].nunique()) #original dataset
print("Cleaned:", df['activity_clean'].nunique()) #activity column after removing spaces, fixing inconsistent letter cases and removing punctutations
print("Grouped:", df['activity_group'].nunique()) #after categorising all the activity under specific keywords

Original: 1608
Cleaned: 1522
Grouped: 9


In [44]:
df[df['activity_group'] == 'other']['activity_clean'].sample(50, random_state=42).tolist()
#sample of 'other' category to improve mapping and grouping


['esso bolivar was torpedoed  shelled by the german submarine u',
 'lifesaving drill',
 'skimboarding',
 'grabbing shark for a selfie',
 'sup foil boarding',
 'playing',
 'waterskiing',
 'washing his feet',
 'shark watching',
 'jet skiing',
 'dangling feet in the water',
 'washing',
 'the ton american freighter  troop transport cape san juan was torpedoed by the japanese submarine i',
 'racing ski',
 'teasing a shark',
 'clamming',
 'jumping in the waves',
 'probabable drowning',
 'crossing the bay at the ford',
 'attempting to anesthetize shark',
 'jumped into river',
 'unknown',
 'lifeguard exercises',
 'holding sharks tail',
 'reaching for life preserver',
 'body found on deserted luxury yacht  christine',
 'paddling',
 'playing on a sandbar',
 'watercraft',
 'jackass teams publicity stunt for discovery channels shark week ',
 'playing',
 'watching the sardine run',
 'kiteboarding',
 'retrieving meat from a cage in the water',
 'burning of the ss missouri',
 'kite boarding',
 'murde

In [46]:
df['activity_group'].value_counts()

activity_group
swimming       1667
surfing        1629
fishing        1355
unknown         585
diving          568
wading          397
other           375
boating         287
catastrophe     187
Name: count, dtype: int64